## (1) Set Environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.system('git clone https://github.com/VlOIET/20242R0136COSE47402.git')

0

In [3]:
!pip install accelerate
!pip install datasets
!pip install peft
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
pip install transformers huggingface-hub

In [5]:
from huggingface_hub import login
from google.colab import userdata
HF_Token = userdata.get('HF_AT')

login(token=HF_Token)

## (2) Preprocessing DATA

In [6]:
from datasets import load_dataset

dataset = load_dataset("Shengtao/recipe")

print(dataset["train"][0])

def preprocess_function(example):
    return {
        "text": f"Question: Show me recipes I can make with {example['ingredients']}. Answer: Title: {example['title']} Directions: {example['directions']}"
    }

processed_dataset = dataset.map(preprocess_function)

train_test_split = processed_dataset["train"].train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]
train_dataset = train_dataset.select(range(10000))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

recipe.csv:   0%|          | 0.00/64.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/32722 [00:00<?, ? examples/s]

{'title': 'Simple Macaroni and Cheese', 'url': 'https://www.allrecipes.com/recipe/238691/simple-macaroni-and-cheese/', 'category': 'main-dish', 'author': 'g0dluvsugly', 'description': "A very quick and easy fix to a tasty side-dish. Fancy, designer mac and cheese often costs forty or fifty dollars to prepare when you have so many exotic and expensive cheeses, but they aren't always the best tasting. This recipe is cheap and tasty.", 'rating': 4.42, 'rating_count': 834, 'review_count': 575, 'ingredients': '1 (8 ounce) box elbow macaroni ; ¼ cup butter ; ¼ cup all-purpose flour ; ½ teaspoon salt ;   ground black pepper to taste ; 2 cups milk ; 2 cups shredded Cheddar cheese', 'directions': 'Bring a large pot of lightly salted water to a boil. Cook elbow macaroni in the boiling water, stirring occasionally until cooked through but firm to the bite, 8 minutes. Drain. Melt butter in a saucepan over medium heat; stir in flour, salt, and pepper until smooth, about 5 minutes. Slowly pour milk 

Map:   0%|          | 0/32722 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="float16")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()

    pad_token_id = tokenizer.pad_token_id
    tokenized["labels"] = [
        [(token if token != pad_token_id else -100) for token in input_ids]
        for input_ids in tokenized["input_ids"]
    ]
    return tokenized

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_val = tokenized_val.remove_columns(["text"])


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6545 [00:00<?, ? examples/s]

## (3) Training

In [9]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)


In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 18


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.816200,0.817837
2,0.791300,0.804810
3,0.782900,0.800397


TrainOutput(global_step=7500, training_loss=0.803532470703125, metrics={'train_runtime': 4187.4049, 'train_samples_per_second': 7.164, 'train_steps_per_second': 1.791, 'total_flos': 6.093193347072e+17, 'train_loss': 0.803532470703125, 'epoch': 3.0})

In [12]:
trainer.save_model("/content/drive/MyDrive/DeepLearning/Final_Project/llama_FT2")
tokenizer.save_pretrained("/content/drive/MyDrive/DeepLearning/Final_Project/llama_FT2")

('/content/drive/MyDrive/DeepLearning/Final_Project/llama_FT2/tokenizer_config.json',
 '/content/drive/MyDrive/DeepLearning/Final_Project/llama_FT2/special_tokens_map.json',
 '/content/drive/MyDrive/DeepLearning/Final_Project/llama_FT2/tokenizer.model',
 '/content/drive/MyDrive/DeepLearning/Final_Project/llama_FT2/added_tokens.json')

In [21]:
input_text = (
    f"You are an expert chef. Based on the given ingredients, suggest a recipe title and step-by-step cooking directions.\n\n"
    f"Ingredients:\n egg pepper \n\n"
    f"Directions:"
)

In [22]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + 300,  # Relative max length
        max_new_tokens=300,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        eos_token_id=tokenizer.eos_token_id
)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Both `max_new_tokens` (=300) and `max_length`(=348) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


You are an expert chef. Based on the given ingredients, suggest a recipe title and step-by-step cooking directions.

Ingredients:
 egg pepper 

Directions:  1. Crack open your eggs into a bowl. Add in some salt to taste. Whisk until well combined. Pour mixture into pan over medium heat. Cook for about five minutes or until slightly set but still runny. Remove from stove top and place onto plate. Sprinkle with black peppers! Serve immediately.
